### requirements for the following codings


In [5]:
### packages required 
!pip install fair-esm 
!pip install torch
!pip install tensorflow
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.0 MB/s 


### peptide embeddings with esm2_t6_8M_UR50D pretrained models
6 layers, 8M parameters, dataset: UR50/D 2021_04, embedding dimension: 320
mode download URL: https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t6_8M_UR50D.pt

In [2]:
def esm_embeddings(peptide_sequence_list):
  # NOTICE: ESM for embeddings is quite RAM usage, if your sequence is too long, 
  #         or you have too many sequences for transformation in a single converting, 
  #         you conputer might automatically kill the job.
  import torch
  import esm
  import collections
  # load the model
  # NOTICE: if the model was not downloaded in your local environment, it will automatically download it.
  model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
  batch_converter = alphabet.get_batch_converter()
  model.eval()  # disables dropout for deterministic results

  # load the peptide sequence list into the bach_converter
  batch_labels, batch_strs, batch_tokens = batch_converter(peptide_sequence_list)
  batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
  ## batch tokens are the embedding results of the whole data set

  # Extract per-residue representations (on CPU)
  with torch.no_grad():
      # Here we export the last layer of the EMS model output as the representation of the peptides
      # model'esm2_t6_8M_UR50D' only has 6 layers, and therefore repr_layers parameters is equal to 6
      results = model(batch_tokens, repr_layers=[6], return_contacts=True)  
  token_representations = results["representations"][6]

  # Generate per-sequence representations via averaging
  # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
  sequence_representations = []
  for i, tokens_len in enumerate(batch_lens):
      sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))
  # save dataset
  # sequence_representations is a list and each element is a tensor
  embeddings_results = collections.defaultdict(list)
  for i in range(len(sequence_representations)):
      # tensor can be transformed as numpy sequence_representations[0].numpy() or sequence_representations[0].to_list
      each_seq_rep = sequence_representations[i].tolist()
      for each_element in each_seq_rep:
          embeddings_results[i].append(each_element)
  embeddings_results = pd.DataFrame(embeddings_results).T
  return embeddings_results

### data loading and embeddings (main dataset)

In [3]:
import numpy as np
import pandas as pd

In [1]:
# training dataset loading
dataset = pd.read_excel('main_ACP_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []

for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('main_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

NameError: ignored

In [7]:
# test dataset loading
dataset = pd.read_excel('main_ACP_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
peptide_sequence_list = []

for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    
embeddings_results = esm_embeddings(peptide_sequence_list)
embeddings_results.to_csv('main_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [8]:
# assign the dataset 
X_train_data_name = 'main_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'main_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [9]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1378, 320)
(344, 320)
(1378,)
(344,)


### data loading and embeddings (alternative dataset)

In [4]:
import numpy as np
import pandas as pd

In [5]:
# training dataset loading
dataset = pd.read_excel('alternative_ACP_train.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 

embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])

embeddings_results.to_csv('alternative_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_train = dataset['label']
y_train = np.array(y_train) # transformed as np.array for CNN model

In [6]:
# test dataset loading
dataset = pd.read_excel('alternative_ACP_test.xlsx',na_filter = False) # take care the NA sequence problem
sequence_list = dataset['sequence'] 
embeddings_results = pd.DataFrame()
# embedding all the peptide one by one
for seq in sequence_list:
    format_seq = [seq,seq] # the setting is just following the input format setting in ESM model, [name,sequence]
    tuple_sequence = tuple(format_seq)
    peptide_sequence_list = []
    peptide_sequence_list.append(tuple_sequence) # build a summarize list variable including all the sequence information
    # employ ESM model for converting and save the converted data in csv format
    one_seq_embeddings = esm_embeddings(peptide_sequence_list)
    embeddings_results= pd.concat([embeddings_results,one_seq_embeddings])
    
embeddings_results.to_csv('alternative_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv')

# loading the y dataset for model development 
y_test = dataset['label']
y_test = np.array(y_test) # transformed as np.array for CNN model

In [7]:
# assign the dataset 
X_train_data_name = 'alternative_ACP_train_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_train_data = pd.read_csv(X_train_data_name,header=0, index_col = 0,delimiter=',')

X_test_data_name = 'alternative_ACP_test_esm2_t6_8M_UR50D_unified_320_dimension.csv'
X_test_data = pd.read_csv(X_test_data_name,header=0, index_col = 0,delimiter=',')

X_train = np.array(X_train_data)
X_test = np.array(X_test_data)

# normalize the X data range
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) # normalize X to 0-1 range 
X_test = scaler.transform(X_test)

In [8]:
# check the dimension of the dataset before model development
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1552, 320)
(388, 320)
(1552,)
(388,)


### Model architecture

In [9]:
def ESM_CNN(X_train, y_train, X_test, y_test):
  from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D
  from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, AveragePooling1D, MaxPooling1D
  from keras.models import Sequential,Model
  from keras.optimizers import SGD
  from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping
  import keras
  from keras import backend as K
  inputShape=(320,1)
  input = Input(inputShape)
  x = Conv1D(128,(3),strides = (1),name='layer_conv1',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool1',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Conv1D(32,(3),strides = (1),name='layer_conv2',padding='same')(input)
  x = BatchNormalization()(x)
  x = Activation('relu')(x)
  x = MaxPooling1D((2), name='MaxPool2',padding="same")(x)
  x = Dropout(0.15)(x)
  x = Flatten()(x)
  x = Dense(64,activation = 'relu',name='fc1')(x)
  x = Dropout(0.15)(x)
  x = Dense(2,activation = 'softmax',name='fc2')(x)
  model = Model(inputs = input,outputs = x,name='Predict')
  # define SGD optimizer
  momentum = 0.5
  sgd = SGD(lr=0.01, momentum=momentum, decay=0.0, nesterov=False)
  # compile the model
  model.compile(loss='sparse_categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])
  # learning deccay setting
  import math
  def step_decay(epoch): # gradually decrease the learning rate 
      initial_lrate=0.1
      drop=0.6
      epochs_drop = 3.0
      lrate= initial_lrate * math.pow(drop,    # math.pow base raised to a power
            math.floor((1+epoch)/epochs_drop)) # math.floor Round numbers down to the nearest integer
      return lrate
  lrate = LearningRateScheduler(step_decay)

  # early stop setting
  early_stop = EarlyStopping(monitor='val_accuracy', patience = 40,restore_best_weights = True)

  # summary the callbacks_list
  callbacks_list = [ lrate , early_stop]

  model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                            epochs=200,callbacks=callbacks_list,batch_size = 8, verbose=1)
  return model, model_history

### 10-fold cross validation

In [10]:
#Implementing 10-fold cross validation
from sklearn.model_selection import KFold
k = 10 
kf = KFold(n_splits=k, shuffle = True, random_state=1)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)

# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []

for train_index , test_index in kf.split(y_train):
    X_train_CV , X_valid_CV = X_train.iloc[train_index,:],X_train.iloc[test_index,:]
    y_train_CV , y_valid_CV = y_train.iloc[train_index] , y_train.iloc[test_index]
    model, model_history = ESM_CNN(X_train_CV, y_train_CV, X_valid_CV, y_valid_CV)
    # confusion matrix 
    predicted_class= []
    predicted_protability = model.predict(X_valid_CV,batch_size=1)
    for i in range(predicted_protability.shape[0]):
      index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
      predicted_class.append(index)
    predicted_class = np.array(predicted_class)
    y_true = y_valid_CV    
    from sklearn.metrics import confusion_matrix
    import math
    # np.ravel() return a flatten 1D array
    TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
    ACC = (TP+TN)/(TP+TN+FP+FN)
    ACC_collecton.append(ACC)
    Sn_collecton.append(TP/(TP+FN))
    Sp_collecton.append(TN/(TN+FP))
    MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
    MCC_collecton.append(MCC)
    BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
    from sklearn.metrics import roc_auc_score
    AUC = roc_auc_score(y_valid_CV, predicted_protability[:,1])
    AUC_collecton.append(AUC)


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
175/175 [==============================] - 3s 12ms/step - loss: 0.9024 - accuracy: 0.8338 - val_loss: 0.3111 - val_accuracy: 0.8718 - lr: 0.1000
Epoch 2/200
175/175 [==============================] - 2s 11ms/step - loss: 0.2342 - accuracy: 0.9062 - val_loss: 0.3234 - val_accuracy: 0.8526 - lr: 0.1000
Epoch 3/200
175/175 [==============================] - 2s 11ms/step - loss: 0.2017 - accuracy: 0.9090 - val_loss: 0.2312 - val_accuracy: 0.8782 - lr: 0.0600
Epoch 4/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1839 - accuracy: 0.9277 - val_loss: 0.2688 - val_accuracy: 0.8718 - lr: 0.0600
Epoch 5/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1700 - accuracy: 0.9298 - val_loss: 0.2265 - val_accuracy: 0.8846 - lr: 0.0600
Epoch 6/200
175/175 [==============================] - 2s 12ms/step - loss: 0.1449 - accuracy: 0.9427 - val_loss: 0.2526 - val_accuracy: 0.8846 - lr: 0.0360
Epoch 7/200
175/175 [==============================] - 2s 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


175/175 [==============================] - 3s 12ms/step - loss: 0.6371 - accuracy: 0.8245 - val_loss: 0.6947 - val_accuracy: 0.4936 - lr: 0.1000
Epoch 2/200
175/175 [==============================] - 2s 12ms/step - loss: 0.2576 - accuracy: 0.8918 - val_loss: 0.5170 - val_accuracy: 0.7500 - lr: 0.1000
Epoch 3/200
175/175 [==============================] - 2s 12ms/step - loss: 0.1934 - accuracy: 0.9126 - val_loss: 0.2144 - val_accuracy: 0.9167 - lr: 0.0600
Epoch 4/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1683 - accuracy: 0.9198 - val_loss: 0.2307 - val_accuracy: 0.9103 - lr: 0.0600
Epoch 5/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1613 - accuracy: 0.9291 - val_loss: 0.2203 - val_accuracy: 0.9295 - lr: 0.0600
Epoch 6/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1537 - accuracy: 0.9420 - val_loss: 0.2283 - val_accuracy: 0.9359 - lr: 0.0360
Epoch 7/200
175/175 [==============================] - 2s 11ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


...

175/175 [==============================] - 3s 12ms/step - loss: 0.5851 - accuracy: 0.8268 - val_loss: 0.8161 - val_accuracy: 0.4581 - lr: 0.1000
Epoch 2/200
175/175 [==============================] - 2s 10ms/step - loss: 0.2547 - accuracy: 0.8991 - val_loss: 0.7292 - val_accuracy: 0.4581 - lr: 0.1000
Epoch 3/200
175/175 [==============================] - 2s 10ms/step - loss: 0.2066 - accuracy: 0.9141 - val_loss: 0.2589 - val_accuracy: 0.9226 - lr: 0.0600
Epoch 4/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1974 - accuracy: 0.9127 - val_loss: 0.1701 - val_accuracy: 0.9484 - lr: 0.0600
Epoch 5/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1817 - accuracy: 0.9241 - val_loss: 0.1489 - val_accuracy: 0.9355 - lr: 0.0600
Epoch 6/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1559 - accuracy: 0.9313 - val_loss: 0.1381 - val_accuracy: 0.9484 - lr: 0.0360
Epoch 7/200
175/175 [==============================] - 2s 10ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


175/175 [==============================] - 3s 12ms/step - loss: 0.8562 - accuracy: 0.8468 - val_loss: 0.6656 - val_accuracy: 0.6387 - lr: 0.1000
Epoch 2/200
175/175 [==============================] - 2s 11ms/step - loss: 0.2396 - accuracy: 0.9019 - val_loss: 0.2875 - val_accuracy: 0.8516 - lr: 0.1000
Epoch 3/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1866 - accuracy: 0.9284 - val_loss: 0.2188 - val_accuracy: 0.8968 - lr: 0.0600
Epoch 4/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1756 - accuracy: 0.9298 - val_loss: 0.2261 - val_accuracy: 0.8839 - lr: 0.0600
Epoch 5/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1631 - accuracy: 0.9406 - val_loss: 0.2419 - val_accuracy: 0.8839 - lr: 0.0600
Epoch 6/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1301 - accuracy: 0.9485 - val_loss: 0.2451 - val_accuracy: 0.8968 - lr: 0.0360
Epoch 7/200
175/175 [==============================] - 2s 11ms/step - 

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


175/175 [==============================] - 3s 12ms/step - loss: 0.6640 - accuracy: 0.8533 - val_loss: 0.5151 - val_accuracy: 0.8323 - lr: 0.1000
Epoch 2/200
175/175 [==============================] - 2s 11ms/step - loss: 0.2396 - accuracy: 0.8984 - val_loss: 0.2927 - val_accuracy: 0.9484 - lr: 0.1000
Epoch 3/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1906 - accuracy: 0.9184 - val_loss: 0.1566 - val_accuracy: 0.9613 - lr: 0.0600
Epoch 4/200
175/175 [==============================] - 2s 11ms/step - loss: 0.1700 - accuracy: 0.9227 - val_loss: 0.1249 - val_accuracy: 0.9548 - lr: 0.0600
Epoch 5/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1664 - accuracy: 0.9284 - val_loss: 0.1338 - val_accuracy: 0.9742 - lr: 0.0600
Epoch 6/200
175/175 [==============================] - 2s 10ms/step - loss: 0.1339 - accuracy: 0.9435 - val_loss: 0.1262 - val_accuracy: 0.9548 - lr: 0.0360
Epoch 7/200
175/175 [==============================] - 2s 11ms/step - 

In [11]:
from statistics import mean, stdev
print(mean(ACC_collecton),'±',stdev(ACC_collecton))
print(mean(BACC_collecton),'±',stdev(BACC_collecton))
print(mean(Sn_collecton),'±',stdev(Sn_collecton))
print(mean(Sp_collecton),'±',stdev(Sp_collecton))
print(mean(MCC_collecton),'±',stdev(MCC_collecton))
print(mean(AUC_collecton),'±',stdev(AUC_collecton))


0.9388089330024814 ± 0.025582379101230845
0.9392731834423343 ± 0.02534071549921132
0.9566038191176267 ± 0.031859440685139635
0.9219425477670421 ± 0.036047696862981544
0.8787425634853295 ± 0.05032148341313998
0.9777547644735536 ± 0.008808004531147924


### model evaluation in test dataset

In [12]:
# result collection list
ACC_collecton = []
BACC_collecton = []
Sn_collecton = []
Sp_collecton = []
MCC_collecton = []
AUC_collecton = []
model, model_history = ESM_CNN(X_train, y_train, X_test , y_test)
# confusion matrix 
predicted_class= []
predicted_protability = model.predict(X_test,batch_size=1)
for i in range(predicted_protability.shape[0]):
  index = np.where(predicted_protability[i] == np.amax(predicted_protability[i]))[0][0]
  predicted_class.append(index)
predicted_class = np.array(predicted_class)
y_true = y_test    
from sklearn.metrics import confusion_matrix
import math
# np.ravel() return a flatten 1D array
TP, FP, FN, TN = confusion_matrix(y_true, predicted_class).ravel() # shape [ [True-Positive, False-positive], [False-negative, True-negative] ]
ACC = (TP+TN)/(TP+TN+FP+FN)
ACC_collecton.append(ACC)
Sn_collecton.append(TP/(TP+FN))
Sp_collecton.append(TN/(TN+FP))
MCC = (TP*TN-FP*FN)/math.pow(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN)),0.5)
MCC_collecton.append(MCC)
BACC_collecton.append(0.5*TP/(TP+FN)+0.5*TN/(TN+FP))
from sklearn.metrics import roc_auc_score
AUC = roc_auc_score(y_test, predicted_protability[:,1])
AUC_collecton.append(AUC)

Epoch 1/200


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


194/194 [==============================] - 3s 13ms/step - loss: 0.6182 - accuracy: 0.8666 - val_loss: 0.6701 - val_accuracy: 0.7294 - lr: 0.1000
Epoch 2/200
194/194 [==============================] - 4s 19ms/step - loss: 0.2327 - accuracy: 0.9085 - val_loss: 0.3029 - val_accuracy: 0.9124 - lr: 0.1000
Epoch 3/200
194/194 [==============================] - 2s 12ms/step - loss: 0.1864 - accuracy: 0.9220 - val_loss: 0.2002 - val_accuracy: 0.9407 - lr: 0.0600
Epoch 4/200
194/194 [==============================] - 2s 10ms/step - loss: 0.1754 - accuracy: 0.9253 - val_loss: 0.1929 - val_accuracy: 0.9459 - lr: 0.0600
Epoch 5/200
194/194 [==============================] - 2s 10ms/step - loss: 0.1660 - accuracy: 0.9227 - val_loss: 0.2082 - val_accuracy: 0.9304 - lr: 0.0600
Epoch 6/200
194/194 [==============================] - 2s 10ms/step - loss: 0.1427 - accuracy: 0.9465 - val_loss: 0.2023 - val_accuracy: 0.9330 - lr: 0.0360
Epoch 7/200
194/194 [==============================] - 2s 10ms/step - 

In [13]:
print(ACC_collecton[0])
print(BACC_collecton[0])
print(Sn_collecton[0])
print(Sp_collecton[0])
print(MCC_collecton[0])
print(AUC_collecton[0])

0.9458762886597938
0.9478874743107267
0.9779005524861878
0.9178743961352657
0.8937615001394401
0.9710915081305133


In [14]:
model.save('ACP_alternative_tensorflow_model',save_format = 'tf') 
!zip -r /content/ACP_alternative_tensorflow_model.zip /content/ACP_alternative_tensorflow_model

  adding: content/ACP_alternative_tensorflow_model/ (stored 0%)
  adding: content/ACP_alternative_tensorflow_model/variables/ (stored 0%)
  adding: content/ACP_alternative_tensorflow_model/variables/variables.index (deflated 64%)
  adding: content/ACP_alternative_tensorflow_model/variables/variables.data-00000-of-00001 (deflated 37%)
  adding: content/ACP_alternative_tensorflow_model/saved_model.pb (deflated 88%)
  adding: content/ACP_alternative_tensorflow_model/assets/ (stored 0%)
  adding: content/ACP_alternative_tensorflow_model/keras_metadata.pb (deflated 89%)
